In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType
import os

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Credit Card Data Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

# Define schema for CSV
schema = StructType([
    StructField("Tanggal", StringType(), True),
    StructField("Provinsi", StringType(), True),
    StructField("Nilai_Server", FloatType(), True),
    StructField("Nilai_Kartu", FloatType(), True),
    StructField("Nilai_Registered", FloatType(), True),
    StructField("Nilai_Unregistered", FloatType(), True)
])

# Try different HDFS paths
hdfs_paths = [
    "hdfs://namenode:8020/data/credit-card-data/DataFixABD.csv",
    "hdfs://namenode:8020/user/jovyan/data/DataFixABD.csv",
    "hdfs://namenode:8020/test.txt",  # Test file we created earlier
]

success = False

# Test HDFS connectivity
print("Testing HDFS connectivity...")
try:
    # Use Hadoop FileSystem API to list files
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path("/"))
    
    print("HDFS root directory contents:")
    for fileStatus in status:
        print(fileStatus.getPath().toString())
    
    # Try to find our CSV file anywhere in the filesystem
    print("\nSearching for CSV files in HDFS...")
    find_cmd = spark.sparkContext._jvm.org.apache.hadoop.fs.FileUtil.stat2Paths(
        fs.globStatus(spark._jvm.org.apache.hadoop.fs.Path("/**/*.csv")))
    
    if find_cmd and len(find_cmd) > 0:
        for path in find_cmd:
            print(f"Found: {path}")
    else:
        print("No CSV files found via glob")
        
except Exception as e:
    print(f"Error testing HDFS: {str(e)}")

# Try reading from each HDFS path
for hdfs_path in hdfs_paths:
    try:
        print(f"\nAttempting to read from: {hdfs_path}")
        df_hdfs = spark.read.csv(hdfs_path, header=True, schema=schema)
        print("Data from HDFS:")
        df_hdfs.show(5)
        print(f"Total records: {df_hdfs.count()}")
        success = True
        break
    except Exception as e:
        print(f"Error reading from {hdfs_path}: {str(e)}")

# If HDFS paths fail, try local file
if not success:
    print("\nTrying local file...")
    try:
        df_local = spark.read.csv("/home/jovyan/work/DataFixABD.csv", 
                              header=True, 
                              schema=schema)
        print("Data from local file:")
        df_local.show(5)
        print(f"Total records: {df_local.count()}")
        
        # If local read successful, try to put it into HDFS
        print("\nAttempting to upload from local file to HDFS...")
        df_local.write.csv("hdfs://namenode:8020/user/jovyan/data/credit_card_data", 
                      header=True, 
                      mode="overwrite")
        print("Successfully wrote data to HDFS")
    except Exception as e2:
        print(f"Error with local file operations: {str(e2)}")

# Stop Spark session
spark.stop()